<a href="https://colab.research.google.com/github/jonathan-arevalo/Data-Analysis-Portfolio/blob/main/HealthInPeru_Malaria%26Dengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descripcion del proyecto

In [ ]:
#vigilancia a malaria y dengue, enfermedades que afectan en sobremanera al peru. Un analisis a como han trabajado en las ultimas dos decadas (o quiza la ultima decada, o incluso una comparacion)
!pip install pyspark
!pip install streamlit

import streamlit as st
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
#from pyspark.sql.functions import "variables"
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np